# Синхронизация с гит

In [13]:
# --- БЛОК №1: НАЧАЛО РАБОТЫ (выполнять в каждой новой сессии) ---

from google.colab import drive, userdata
import os

# 1. Монтируем Google Drive
drive.mount('/content/drive')

# 2. Получаем секретный токен
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

# 3. Указываем ваше имя и email (как на GitHub)
GIT_USERNAME = "kirichich1" # <-- Ваше имя пользователя
GIT_EMAIL = "kirichich@bk.ru" # <-- Ваша почта

# 4. !!! ПРЕДСТАВЛЯЕМСЯ СИСТЕМЕ GIT !!!
# Эта конфигурация будет действовать до конца текущей сессии
!git config --global user.name "{GIT_USERNAME}"
!git config --global user.email "{GIT_EMAIL}"

# 5. Переходим в директорию проекта
PROJECT_PATH = "/content/drive/MyDrive/GitHub/no2_prediction_pipeline"
%cd {PROJECT_PATH}

# 6. (Рекомендуется) Скачиваем последние изменения с GitHub на случай, если вы работали с другого устройства
!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/no2_prediction_pipeline
Already up to date.


# Импорт нужных инструментов


In [1]:
import ee
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ee.Authenticate()
ee.Initialize(project='pollutionmlproject')

In [3]:
!pip install earthengine-api

# Выгрузка данных

In [4]:
# --- ПАРАМЕТРЫ ГОРОДА ---
CITY_GEOM = ee.Geometry.Point(73.5, 61.25).buffer(25000)
CITY_NAME = 'Surgut'

# ==============================================================================
# == УСТАНОВИТЕ ГОД ДЛЯ ВЫГРУЗКИ ===============================================
# ==============================================================================
YEAR_TO_DOWNLOAD = 2019 # <--- МЕНЯЙТЕ ГОД ЗДЕСЬ
# ==============================================================================

# --- Параметры фильтрации S5P ---
PIXEL_CLOUD_THRESHOLD = 0.5
SOLAR_ZENITH_ANGLE_THRESHOLD = 80.0

# ==============================================================================
# == БЛОК 2: ФУНКЦИИ-СБОРЩИКИ ДАННЫХ (S5P обновлена для AAI) ===================
# ==============================================================================

def get_s5p_data_for_date(target_date, geometry):
    """
    Получает данные NO2 и AAI за одну дату.
    КРИТИЧЕСКИ ВАЖНО: Маска облачности берется из NO2 и применяется к AAI.
    """
    try:
        start = ee.Date(target_date)
        end = start.advance(1, 'day')

        # 1. Запрашиваем коллекцию NO2 (источник маски облачности)
        s5p_no2_coll = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
            .filterBounds(geometry).filterDate(start, end)

        # 2. Запрашиваем коллекцию AAI (данные, которые нужно отфильтровать)
        s5p_aai_coll = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_AER_AI') \
            .filterBounds(geometry).filterDate(start, end)

        # Если нет снимка NO2, мы не можем ни получить NO2, ни сделать маску для AAI.
        if s5p_no2_coll.size().getInfo() == 0:
            return None

        # Берем средние снимки за день
        image_no2 = s5p_no2_coll.mean()

        # Проверяем, есть ли AAI. Если нет, создаем пустую переменную, чтобы код не упал
        has_aai = s5p_aai_coll.size().getInfo() > 0
        image_aai = s5p_aai_coll.mean() if has_aai else ee.Image.constant(None)

        # --- СОЗДАНИЕ МАСКИ (по данным NO2) ---
        cloud_mask = image_no2.select('cloud_fraction').lt(PIXEL_CLOUD_THRESHOLD)
        sza_mask = image_no2.select('solar_zenith_angle').lt(SOLAR_ZENITH_ANGLE_THRESHOLD)
        combined_mask = cloud_mask.And(sza_mask)

        # --- ПРИМЕНЕНИЕ МАСКИ К ОБОИМ ПРОДУКТАМ ---
        # Фильтруем NO2
        no2_filtered = image_no2.updateMask(combined_mask)

        # Фильтруем AAI той же самой маской! (Реализация вашего плана)
        aai_filtered = image_aai.updateMask(combined_mask) if has_aai else image_aai

        # Сбор статистики (Reduce Region)
        stats_no2 = no2_filtered.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=1000,
            maxPixels=1e9
        ).getInfo()

        # Отдельно собираем облачность (без маски облачности, чтобы знать реальное покрытие)
        cloud_stats = image_no2.select('cloud_fraction').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=1000,
            maxPixels=1e9
        ).getInfo()

        stats_aai = {}
        if has_aai:
            stats_aai = aai_filtered.select('absorbing_aerosol_index').reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e9
            ).getInfo()

        return {
            'no2_trop_mean': stats_no2.get('tropospheric_NO2_column_number_density'),
            'cloud_frac_mean': cloud_stats.get('cloud_fraction'),
            'aai_mean': stats_aai.get('absorbing_aerosol_index') if has_aai else None
        }

    except Exception as e:
        print(f"Ошибка S5P на дате {target_date}: {e}")
        return None

def get_era5_data_for_date(target_date, geometry):
    """
    Функция ERA5 без изменений.
    """
    try:
        start = ee.Date(target_date)
        end = start.advance(1, 'day')
        era5_bands = ['temperature_2m', 'surface_pressure',
                      'u_component_of_wind_10m', 'v_component_of_wind_10m']

        image = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
                  .filterBounds(geometry) \
                  .filterDate(start, end) \
                  .select(era5_bands) \
                  .mean()

        if image:
            data = image.reduceRegion(
                reducer=ee.Reducer.mean(), geometry=geometry, scale=11132, maxPixels=1e9
            ).getInfo()

            temp_celsius = data.get('temperature_2m') - 273.15 if data.get('temperature_2m') else None
            pressure_hpa = data.get('surface_pressure') / 100 if data.get('surface_pressure') else None

            return {
                'temperature_celsius': temp_celsius,
                'pressure_hpa': pressure_hpa,
                'u_wind_10m': data.get('u_component_of_wind_10m'),
                'v_wind_10m': data.get('v_component_of_wind_10m')
            }
        return None
    except Exception as e:
        print(f"Ошибка ERA5 на дате {target_date}: {e}"); return None

# ==============================================================================
# == БЛОК 3: ОСНОВНОЙ ЦИКЛ СБОРА ДАННЫХ =========================================
# ==============================================================================

all_data = []
print(f"Начинаю сбор данных (NO2 + AAI + ERA5) для города: {CITY_NAME} за {YEAR_TO_DOWNLOAD} год")

start_of_period = f"{YEAR_TO_DOWNLOAD}-02-01"
end_of_period = f"{YEAR_TO_DOWNLOAD}-09-30"
print(f"--- Период: с {start_of_period} по {end_of_period} ---")
date_range = pd.date_range(start=start_of_period, end=end_of_period, freq='D')

for date in tqdm(date_range, desc=f"Год {YEAR_TO_DOWNLOAD}"):
    current_date_str = date.strftime('%Y-%m-%d')

    # Теперь эта функция возвращает и NO2, и AAI
    s5p_data = get_s5p_data_for_date(current_date_str, CITY_GEOM)
    era5_data = get_era5_data_for_date(current_date_str, CITY_GEOM)

    row = {'date': date, 'city': CITY_NAME}

    if s5p_data:
        row.update(s5p_data)
    if era5_data:
        row.update(era5_data)

    all_data.append(row)

# ==============================================================================
# == БЛОК 4: ФОРМИРОВАНИЕ И СОХРАНЕНИЕ ИТОГОВОГО ФАЙЛА ========================
# ==============================================================================
final_df = pd.DataFrame(all_data)

# Обновленный порядок колонок, включая AAI
column_order = [
    'date', 'city',
    'no2_trop_mean', 'aai_mean', 'cloud_frac_mean', # Добавил aai_mean сюда
    'temperature_celsius', 'pressure_hpa', 'u_wind_10m', 'v_wind_10m'
]

# Пересортировка колонок (с защитой, если вдруг данных не было вообще и колонка не создалась)
existing_cols = [col for col in column_order if col in final_df.columns]
final_df = final_df[existing_cols]

# Заполнение пропусков AAI нулями (согласно вашему Плану Шаг 2)
if 'aai_mean' in final_df.columns:
    final_df['aai_mean'] = final_df['aai_mean'].fillna(0)
else:
    # Если за весь период не нашлось ни одного пикселя AAI, создаем колонку с нулями
    final_df['aai_mean'] = 0

output_filename = f'{CITY_NAME}_data_{YEAR_TO_DOWNLOAD}_Feb-Sep_WITH_AAI.csv'
final_df.to_csv(output_filename, index=False)

print(f"\nСбор данных завершен.")
print(f"Данные по AAI отфильтрованы по маске облачности NO2 (< {PIXEL_CLOUD_THRESHOLD}).")
print(f"Пропуски в AAI заполнены нулями.")
print(f"Файл сохранен: {output_filename}")

Начинаю сбор данных (NO2 + AAI + ERA5) для города: Surgut за 2019 год
--- Период: с 2019-02-01 по 2019-09-30 ---


Год 2019:   0%|          | 0/242 [00:00<?, ?it/s]


Сбор данных завершен.
Данные по AAI отфильтрованы по маске облачности NO2 (< 0.5).
Пропуски в AAI заполнены нулями.
Файл сохранен: Surgut_data_2019_Feb-Sep_WITH_AAI.csv


In [8]:
final_df.head(50)

,date,city,no2_trop_mean,aai_mean,cloud_frac_mean,temperature_celsius,pressure_hpa,u_wind_10m,v_wind_10m
0,2019-02-01,Surgut,0.000033,-0.875213,0.358617,-41.978251,1038.040702,-2.045977,0.024103
1,2019-02-02,Surgut,0.000033,-1.112367,0.234913,-39.245128,1034.768421,-1.990192,-1.203328
2,2019-02-03,Surgut,0.000037,-1.398887,0.269149,-38.905555,1037.783115,-0.378380,-1.401940
3,2019-02-04,Surgut,0.000030,-1.116266,0.371478,-33.816355,1038.234460,0.179797,-1.944240
4,2019-02-05,Surgut,0.000037,-0.972759,0.423403,-33.523440,1041.622638,-2.200455,-0.714484
5,2019-02-06,Surgut,0.000028,-1.005650,0.318854,-36.704566,1041.708829,-2.058211,0.023261
6,2019-02-07,Surgut,NaN,-0.811202,0.572168,-35.923282,1039.938504,-1.397884,1.266828
7,2019-02-08,Surgut,NaN,-0.870119,0.705313,-28.662623,1030.883290,-1.427336,4.284851
8,2019-02-09,Surgut,NaN,0.000000,0.902351,-24.289176,1023.343677,-1.561984,3.770464
9,2019-02-10,Surgut,0.000033,-0.877948,0.234154,-24.022794,1018.162678,-0.483065,2.485641


In [6]:
final_df['no2_trop_mean'].isna().sum()

np.int64(76)

In [7]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 242 non-null    datetime64[ns]
 1   city                 242 non-null    object        
 2   no2_trop_mean        166 non-null    float64       
 3   aai_mean             242 non-null    float64       
 4   cloud_frac_mean      242 non-null    float64       
 5   temperature_celsius  242 non-null    float64       
 6   pressure_hpa         242 non-null    float64       
 7   u_wind_10m           242 non-null    float64       
 8   v_wind_10m           242 non-null    float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 17.1+ KB


In [10]:
final_df['no2_trop_mean'].isna().sum()

np.int64(76)

# Коммит для гит

In [15]:
# --- БЛОК №2: СОХРАНЕНИЕ РЕЗУЛЬТАТОВ (в конце работы) ---

# 1. Проверяем статус (опционально)
!git status

# 2. ДОБАВЛЯЕМ файлы в "коробку" для отправки. Это то, что вы пропустили.
!git add .

# 3. ПОДПИСЫВАЕМ "коробку" (делаем коммит)
!git commit -m "feat: Add script for AAI data export"

# 4. ОТПРАВЛЯЕМ "коробку" на GitHub
!git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   notebooks/data_export_Surgut_with_AAI.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/raw/add_predictors/

no changes added to commit (use "git add" and/or "git commit -a")
[main 9356dae] feat: Add script for AAI data export
 2 files changed, 244 insertions(+), 1 deletion(-)
 create mode 100644 data/raw/add_predictors/Surgut_data_2019_Feb-Sep_WITH_AAI.csv
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 17.60 KiB | 643.00 KiB/s, done.
Total 8 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/ki